In [10]:
import pandas
from fpgrowth_py import fpgrowth
import pandas as pd
import re
import json
import spacy
import seaborn as sns
from spacy import displacy
from flashtext import KeywordProcessor
import ahocorasick
import nltk
# import pyplot-express as px

pathprefix = r"D:\Code Working Area\Jupyter\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData"
jsfile = pathprefix+"\\Transactions.json"

project = pathprefix+"\\Gg.csv"
risk = pathprefix+"\\Risk_Simplified.xlsx"
stake = pathprefix+"\\expansive.csv"

project1 = pathprefix+"\\newTitle_Project.xlsx"
risk0 = pathprefix+"\\RiskFinal.xlsx"
stake1 = pathprefix+"\\New_StakeHolder_Abstract.xlsx"

stake2 = pathprefix+"\\second_layer_stakeholder.xlsx"

In [2]:
# !pip install regex
# !pip install sapcy
# !pip install fpgrowth_py
# !pip install pandas
# !pip install seaborn
# !pip install openpyxl
# !python -m spacy install en_core_web_sm
stake2

'D:\\Code Working Area\\Jupyter\\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\\ExcelData\\second_layer_stakeholder.xlsx'

In [2]:
pj = pd.read_csv(project, sep = ",")
risk1 = pd.read_excel(risk)
stk = pd.read_csv(stake, sep = ",")

prj = pd.read_excel(project1)
risk2 = pd.read_excel(risk0)
stk1 = pd.read_excel(stake1)
stk2 = pd.read_excel(stake2)
stk2.dropna(inplace=True)

en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

C:\Users\jqi22\AppData\Local\Temp\ipykernel_40404\1512847667.py:1: DtypeWarning: Columns (3,7,8,12,13,18,27,28,29,30,31,38,39,40,41,42,43,44,45,50,51,53,54,59,63,64,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  pj = pd.read_csv(project, sep = ",")


In [3]:
# get the target text from original dataset to match
nproject = pd.DataFrame(pj["Article Title"])
nrisk = pd.DataFrame(risk1["Abstract"])
nstack = pd.DataFrame(stk["Abstract"])

nrisk.dropna(inplace=True, how="any")

stk1.name = stk1.name.str.lower()

In [4]:
def reload_dashes(droped: int)->pd.DataFrame:
    dashes = pd.read_excel(pathprefix+"\\adjustment.xlsx")
    dashes = dashes[dashes.frequency > droped].Words.to_list()
    return set(dashes)

stopwords |= reload_dashes(2)
# filout = pd.read_excel(r"D:\Code Working Area\Python\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData\filtered.xlsx")
# stopwords |= set(filout.name.to_list())
stopwords |= set([str(num) for num in range(1,100)])

# manually add the words to the stopwords
possiblew = {"connections", "efficacy", "life", "This"}
stopwords |= possiblew

In [5]:
risk2.Abstract=risk2.Abstract.astype(str)
stk2.stk=stk2.stk.astype(str)

In [6]:
keypro=KeywordProcessor()
keypro.add_keywords_from_list(prj["Article Title"].to_list()+risk2["Abstract"].to_list()+list(set(stk2["stk"].to_list())))

In [7]:
def abst(args:str):
    return keypro.extract_keywords(args)
res=nrisk.Abstract.apply(abst)
res=res.apply(lambda x: list(set(x)))

In [8]:
res=res.apply(lambda x: list(set(x)))
temp=res.explode(ignore_index=True).dropna(how="any")

In [58]:
tempres=temp.value_counts().to_frame()
tempres["words"]=tempres.index
tempres.reset_index(drop=True, inplace=True)

In [59]:
tempres["word type"]=tempres.words.apply(lambda row: nltk.pos_tag(nltk.word_tokenize(row))[0][1])

In [74]:
wordstp=tempres["word type"].value_counts().index.to_list()
with pd.ExcelWriter(pathprefix+"\\words_types.xlsx") as writer:
    for val in wordstp:
        tempres[tempres["word type"]==val].to_excel(writer,index=False, sheet_name=val)

In [71]:
tempres[tempres["word type"]=="NNP"].to_excel(pathprefix+"\\words_type.xlsx", sheet_name="NNP")

['NN',
 'NNP',
 'DT',
 'JJ',
 'VBG',
 'NNS',
 'VBN',
 'CD',
 'RB',
 'IN',
 'JJR',
 'JJS',
 'VB',
 'VBZ',
 'PRP',
 'CC',
 'WRB',
 'WP',
 'PRP$',
 'VBD',
 'TO',
 'RBR',
 'MD',
 '$',
 'WDT',
 'NNPS']